In [57]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import torch.nn.functional as F

In [58]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_data = datasets.ImageFolder('C:\\Users\\srisi\\OneDrive\\Desktop\\sidxt\\hub\\FractureCNN\\kaggle\\dataset\\train', transform=transform)
test_data = datasets.ImageFolder('C:\\Users\\srisi\\OneDrive\\Desktop\\sidxt\\hub\\FractureCNN\\kaggle\\dataset\\test', transform=transform)
val_data = datasets.ImageFolder('C:\\Users\\srisi\\OneDrive\\Desktop\\sidxt\\hub\\FractureCNN\\kaggle\\dataset\\val', transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=32, shuffle=True)


In [59]:
conv1 = nn.Conv2d(3, 32, 3)
conv2 = nn.Conv2d(32, 64, 3)
conv3 = nn.Conv2d(64, 128, 3)
pool = nn.MaxPool2d(2)
fc1 = nn.Linear(128 * 26 * 26, 256)
fc2 = nn.Linear(256, 128)
fc3 = nn.Linear(128, 1)
dropout = nn.Dropout(0.3)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
conv1.to(device)
conv2.to(device)
conv3.to(device)
fc1.to(device)
fc2.to(device)
fc3.to(device)
pool.to(device)
dropout.to(device)


def forward(x):
    x = pool(F.leaky_relu(conv1(x)))
    x = dropout(x)
    x = pool(F.leaky_relu(conv2(x)))
    x = dropout(x)
    x = pool(F.leaky_relu(conv3(x)))
    x = dropout(x)
    x = x.view(-1, 128 * 26 * 26)
    x = F.leaky_relu(fc1(x))
    x = dropout(x)
    x = F.leaky_relu(fc2(x))
    x = dropout(x)
    x = torch.sigmoid(fc3(x))
    return x
criterion = nn.BCELoss()
optimizer = optim.Adam(list(conv1.parameters()) + list(conv2.parameters()) + list(conv3.parameters()) + 
                       list(fc1.parameters()) + list(fc2.parameters()) + list(fc3.parameters()), lr=0.001)

In [60]:
for epoch in range(10):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = forward(inputs)
        loss = criterion(outputs, labels.unsqueeze(1).float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"epoch {epoch+1}, loss: {running_loss / len(train_loader)}")


epoch 1, loss: 0.6379364832889202
epoch 2, loss: 0.3386526398591755
epoch 3, loss: 0.1247135673389959
epoch 4, loss: 0.07307162957876026
epoch 5, loss: 0.04974293106667835
epoch 6, loss: 0.022513704605232707
epoch 7, loss: 0.030153310708020267
epoch 8, loss: 0.02200262060954304
epoch 9, loss: 0.02526866614965791
epoch 10, loss: 0.029569357750457242
epoch 11, loss: 0.022202633607251682
epoch 12, loss: 0.011619586009027622
epoch 13, loss: 0.0024160115118609456
epoch 14, loss: 0.0027869555983533446
epoch 15, loss: 0.02798095095450533


In [61]:
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = forward(inputs)
        predicted = torch.round(outputs)
        total += labels.size(0)
        correct += (predicted == labels.unsqueeze(1)).sum().item()

print(f"test accuracy: {100 * correct / total}%")

test accuracy: 98.49624060150376%


In [62]:
torch.save({
    'conv1_state_dict': conv1.state_dict(),
    'conv2_state_dict': conv2.state_dict(),
    'conv3_state_dict': conv3.state_dict(),
    'fc1_state_dict': fc1.state_dict(),
    'fc2_state_dict': fc2.state_dict(),
    'fc3_state_dict': fc3.state_dict(),
    'optimizer_state_dict': optimizer.state_dict()
}, 'main.pth')

In [63]:
from sklearn.metrics import precision_recall_fscore_support
all_labels = []
all_predictions = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = forward(inputs)
        predicted = torch.round(outputs)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='binary')

print(f"Test Accuracy: {100 * correct / total}%")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


Test Accuracy: 98.49624060150376%
Precision: 0.99
Recall: 0.99
F1 Score: 0.99
